In [2]:
# header files loaded
import numpy as np
import cv2
import glob
from ReadCameraModel import *
from UndistortImage import *

In [3]:
files = glob.glob("/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/*")
files = sorted(files)
print(files)

['/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381444704913.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381444767404.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381444829896.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381444892384.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381444954879.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381445017371.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381445079860.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381445142354.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381445204845.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381445267335.png', '/home/arpitdec5/Desktop/structure_from_motion/data/stereo/centre/1399381445329828.png', '/home/arpitdec5/Des

In [22]:
# get the image
def get_image(file):
    """
    Inputs:
    
    file: the file to be read
    
    Outputs:
    
    image: numpy array representing the image
    """
    
    image = cv2.imread(file, 0)
    image = cv2.cvtColor(image, cv2.COLOR_BayerGR2BGR)
    fx, fy, cx, cy, camera_image, LUT = ReadCameraModel("../data/model/")
    k_matrix = np.zeros((3, 3))
    k_matrix[0, 0] = fx
    k_matrix[1, 1] = fy
    k_matrix[2, 2] = 1
    k_matrix[0, 2] = cx
    k_matrix[1, 2] = cy
    image = UndistortImage(image, LUT)
    return (image, k_matrix)


#  get transformation matrices
def get_transformation_matrix(ptsLeft, ptsRight):
    
    # mean for ptsLeft and ptsRight
    ptsLeft_mean_x = np.mean(ptsLeft[:, 0])
    ptsLeft_mean_y = np.mean(ptsLeft[:, 1])
    ptsRight_mean_x = np.mean(ptsRight[:, 0])
    ptsRight_mean_y = np.mean(ptsRight[:, 1])
    
    # get updated ptsLeft and ptsRight
    sum_ptsLeft = 0.0
    sum_ptsRight = 0.0
    for index in range(0, len(ptsLeft)):
        x = ptsLeft[index][0] - ptsLeft_mean_x
        y = ptsLeft[index][1] - ptsLeft_mean_y
        sum_ptsLeft = sum_ptsLeft + np.sqrt((x ** 2) + (y ** 2))
        
    for index in range(0, len(ptsRight)):
        x = ptsRight[index][0] - ptsRight_mean_x
        y = ptsRight[index][1] - ptsRight_mean_y
        sum_ptsRight = sum_ptsRight + np.sqrt((x ** 2) + (y ** 2))
    sum_ptsLeft = sum_ptsLeft / len(ptsLeft)
    sum_ptsRight = sum_ptsRight / len(ptsRight)
    
    # scale factor for ptsLeft and ptsRight
    scale_ptsLeft = np.sqrt(2) / sum_ptsLeft
    scale_ptsRight = np.sqrt(2) / sum_ptsRight
    
    # get transformation matrices
    ptsLeft_transformation_matrix = np.dot(np.array([[scale_ptsLeft, 0, 0], [0, scale_ptsLeft, 0], [0, 0, 1]]), np.array([[1, 0, -ptsLeft_mean_x], [0, 1, -ptsLeft_mean_y], [0, 0, 1]]))
    ptsRight_transformation_matrix = np.dot(np.array([[scale_ptsRight, 0, 0], [0, scale_ptsRight, 0], [0, 0, 1]]), np.array([[1, 0, -ptsRight_mean_x], [0, 1, -ptsRight_mean_y], [0, 0, 1]]))
    
    # get normalized points
    for index in range(0, len(ptsLeft)):
        point = np.array([[ptsLeft[index][0]], [ptsLeft[index][1]], [1]])
        point = np.dot(ptsLeft_transformation_matrix, point)
        ptsLeft[index] = np.array([point[0][0] / point[2][0], point[1][0] / point[2][0]])
    
    for index in range(0, len(ptsRight)):
        point = np.array([[ptsRight[index][0]], [ptsRight[index][1]], [1]])
        point = np.dot(ptsRight_transformation_matrix, point)
        ptsRight[index] = np.array([point[0][0] / point[2][0], point[1][0] / point[2][0]])
    
    # return matrices
    return (ptsLeft, ptsRight, ptsLeft_transformation_matrix, ptsRight_transformation_matrix)


# get keypoints between frame 1 and frame 2
def get_keypoints(image1, image2):
    """
    Inputs:
    
    image1: left image
    image2: right image
    
    Outputs:
    
    ptsLeft: point correspondences for left image
    ptsRight: point correspondences for right image
    """
    
    # use sift keypoint to get the points
    sift = cv2.ORB_create()
    kp1, des1 = sift.detectAndCompute(image1, None)
    kp2, des2 = sift.detectAndCompute(image2, None)
    des1 = np.asarray(des1, np.float32)
    des2 = np.asarray(des2, np.float32)
    flann = cv2.FlannBasedMatcher(dict(algorithm = 0, trees = 5), dict(checks = 50))
    matches = flann.knnMatch(des1, des2, k = 2)
    good = []
    ptsLeft = []
    ptsRight = []

    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            ptsRight.append(kp2[m.trainIdx].pt)
            ptsLeft.append(kp1[m.queryIdx].pt)

    ptsLeft = np.int32(ptsLeft)
    ptsRight = np.int32(ptsRight)
    return (ptsLeft, ptsRight)


# get fundamental matrix with ransac
def get_fundamental_matrix_ransac(ptsLeft, ptsRight):
    """
    Inputs:
    
    ptsLeft: array of 8 points for left image
    ptsRight: array of 8 points for right image
    
    Outputs:
    
    fundamental_mat: fundamental matrix of size (3 x 3)
    """
    
    # normalise points
    (ptsLeft, ptsRight, ptsLeft_transformation_matrix, ptsRight_transformation_matrix) = get_transformation_matrix(ptsLeft, ptsRight)

    # ransac for better matrix estimation
    iterations = 5000
    threshold = 0.03
    count = 0
    best_fundamental_matrix = get_fundamental_matrix(ptsLeft[:8], ptsRight[:8], ptsLeft_transformation_matrix, ptsRight_transformation_matrix)
    for iteration in range(0, iterations):
        
        indexes = np.random.choice(np.array(ptsLeft).shape[0], 8, replace=False)
        selected_ptsLeft = []
        selected_ptsRight = []
        for index in range(0, len(ptsLeft)):
            flag = 0
            for k in range(0, len(indexes)):
                if(index == indexes[k]):
                    flag = 1
                    break
            
            if(flag):
                selected_ptsLeft.append(ptsLeft[index])
                selected_ptsRight.append(ptsRight[index])
    
        estimated_fundamental_mat = get_fundamental_matrix(selected_ptsLeft, selected_ptsRight, ptsLeft_transformation_matrix, ptsRight_transformation_matrix)
        estimated_count = 0
        for index in range(0, len(ptsLeft)):
            x_right = np.array([[ptsRight[index][0]], [ptsRight[index][1]], [1]])
            x_left = np.array([[ptsLeft[index][0]], [ptsLeft[index][1]], [1]])
            error = np.dot(x_right.T, np.dot(estimated_fundamental_mat, x_left))
            
            if(np.abs(error) < threshold):
                estimated_count = estimated_count + 1
                
        if(estimated_count > count):
            count = estimated_count
            best_fundamental_matrix = estimated_fundamental_mat
    
    # return fundamental matrix
    return best_fundamental_matrix
    
    
# get fundamental matrix
def get_fundamental_matrix(pointsLeft, pointsRight, ptsLeft_transformation_matrix, ptsRight_transformation_matrix):
    """
    Inputs:
    
    pointsLeft: array of 8 points for left image
    pointsRight: array of 8 points for right image
    ptsLeft_transformation_matrix: transformation matrix for left image
    ptsRight_transformation_matrix: transformation matrix for right image
    
    Outputs:
    
    fundamental_mat: fundamental matrix of size (3 x 3)
    """
    
    # construct a matrix
    a_matrix = []
    for index in range(0, len(pointsLeft)):
        x, y = pointsLeft[index][0], pointsLeft[index][1]
        u, v = pointsRight[index][0], pointsRight[index][1]
        a_matrix.append([x * u, y * u, u, x * v, y * v, v, x, y, 1])
    
    # svd of A
    a_matrix = np.array(a_matrix)
    u, s, vh = np.linalg.svd(a_matrix)
        
    # compute fundamental_mat
    vh = vh.T
    fundamental_mat = vh[:, -1]
    fundamental_mat = fundamental_mat.reshape((3, 3))
    
    # enforce rank 2 constraint and update fundamental_mat
    u, s, vh = np.linalg.svd(fundamental_mat)
    s[2] = 0
    fundamental_mat = np.dot(u, np.dot(np.diag(s), vh))
    
    # un-normalize fundamental_mat
    fundamental_mat = np.dot(ptsRight_transformation_matrix.T, np.dot(fundamental_mat, ptsLeft_transformation_matrix))
    
    # return the matrix
    return fundamental_mat


# estimate essential matrix
def get_essential_matrix(fundamental_matrix, k_matrix):
    """
    Inputs:
    
    fundamental_matrix: Matrix that relates image coordinates in one image to the other
    k_matrix: the calibration matrix of the camera
    
    Outputs:
    
    essential_matrix: return essential matrix
    """
    
    essential_matrix = np.dot(k_matrix.T, np.dot(fundamental_matrix, k_matrix))
    u, s, vh = np.linalg.svd(essential_matrix)
    s[0] = 1
    s[1] = 1
    s[2] = 0
    essential_matrix = np.dot(u, np.dot(s, vh))
    essential_matrix = essential_matrix / np.linalg.norm(essential_matrix)
    
    # return matrix
    return essential_matrix

In [25]:
count = 0
for index in range(0, len(files) - 1, 2):
    
    # get two images
    (image1, k_matrix) = get_image(files[index])
    (image2, k_matrix) = get_image(files[index+1])
    
    # get keypoints
    (ptsLeft, ptsRight) = get_keypoints(image1, image2)
    
    # get fundamental matrix
    fundamental_matrix = get_fundamental_matrix_ransac(ptsLeft.copy(), ptsRight.copy())
    
    # get essential matrix
    essential_matrix = get_essential_matrix(fundamental_matrix, k_matrix)
    
    count = count + 1

0
272
()
()
0
315
()
()
74
74
()
()
275
275
()
()
0
24
()
()
23
23
()
()
310
310
()
()
272
272
()
()
73
73
()
()
233
233
()
()
242
242
()
()
159
159
()
()
245
245
()
()
229
229
()
()
191
191
()
()
210
210
()
()
218
218
()
()
212
212
()
()
188
188
()
()
210
210
()
()
197
197
()
()
193
193
()
()
171
171
()
()
181
181
()
()
172
172
()
()
147
147
()
()
158
158
()
()
154
154
()
()
150
150
()
()
140
140
()
()
139
139
()
()
119
119
()
()
102
102
()
()
117
117
()
()
95
95
()
()
114
114
()
()
94
94
()
()
90
90
()
()
96
96
()
()
101
101
()
()
102
102
()
()
100
100
()
()
101
101
()
()
123
123
()
()
122
122
()
()
112
112
()
()
124
124
()
()
173
173
()
()
165
165
()
()
221
221
()
()
204
204
()
()
226
226
()
()
257
257
()
()
226
226
()
()
272
272
()
()
250
250
()
()
262
262
()
()
262
262
()
()
255
255
()
()
248
248
()
()
268
268
()
()
268
268
()
()
281
281
()
()
260
260
()
()
236
236
()
()
222
222
()
()
198
198
()
()
248
248
()
()
199
199
()
()
220
220
()
()
110
110
()
()
143
143
()
()
250
250
()
()